In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

Import Model

In [2]:
UnWeighted_Model=keras.models.load_model('drive/MyDrive/CNNSpeciesAcc0.9133val0.8493')

Import Data

In [2]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [3]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [4]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [5]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

Get val data probabilities

In [7]:
y_proba=UnWeighted_Model.predict(X_val)

In [8]:
y_proba[1]

array([3.6509544e-01, 5.0902392e-05, 1.5057314e-02, 1.4492625e-06,
       8.8719971e-13, 2.0239108e-06, 1.9344385e-04, 7.9907929e-12,
       3.2738722e-06, 3.8893751e-05, 6.2414634e-22, 9.8531291e-07,
       5.1459068e-01, 1.1715752e-02, 5.5191405e-05, 5.8600608e-10,
       2.8074014e-06, 9.1593608e-04, 1.9018884e-06, 6.6723888e-10,
       1.2768989e-14, 6.2570500e-12, 1.8522213e-08, 1.6897484e-07,
       9.2255339e-02, 1.5063314e-07, 3.5252967e-09, 1.8258364e-05],
      dtype=float32)

In [9]:
# Turn one hot array into integers
y_val_1d=np.argmax(y_val, axis=1)
y_predictions=np.argmax(y_proba, axis=1)

In [10]:
y_dataframe=pd.DataFrame({'y_predictions': y_predictions,
                          'y_actual': y_val_1d
                          })

In [11]:
y_dataframe

,y_predictions,y_actual
0,25,25
1,12,12
2,2,2
3,23,23
4,9,14
...,...,...
8268,1,1
8269,13,13
8270,1,1
8271,1,1


In [16]:
y_proba_list=[]

for each in y_proba:
  y_proba_list.append(each)

In [14]:
#y_dataframe['y_proba']=y_proba_list

In [17]:
#round to 5 decimals

rounded_proba_list=[]
for each_array in y_proba_list:
  new_array=[]
  for each_number in each_array:
    rounded_number=round(each_number, ndigits=5)
    new_array.append('{0:6f}'.format(rounded_number))
  rounded_proba_list.append(new_array)

In [18]:
y_dataframe['y_rounded_proba']=rounded_proba_list

In [19]:
y_dataframe.head(5)

,y_predictions,y_actual,y_rounded_proba
0,25,25,"[0.000000, 0.000000, 0.053620, 0.000000, 0.000..."
1,12,12,"[0.365100, 0.000050, 0.015060, 0.000000, 0.000..."
2,2,2,"[0.000000, 0.000000, 0.842370, 0.000000, 0.000..."
3,23,23,"[0.000000, 0.000000, 0.000000, 0.000000, 0.000..."
4,9,14,"[0.010330, 0.000220, 0.002390, 0.050820, 0.000..."


In [29]:
#y_dataframe['y_rounded_proba'][15]

In [20]:
## locate incorrectly classified animals
incorrect_df=y_dataframe.loc[(y_dataframe['y_predictions'])!=(y_dataframe['y_actual'])]

In [21]:
incorrect_df

,y_predictions,y_actual,y_rounded_proba
4,9,14,"[0.010330, 0.000220, 0.002390, 0.050820, 0.000..."
6,0,13,"[0.590350, 0.000450, 0.083620, 0.000120, 0.000..."
8,13,12,"[0.011090, 0.000720, 0.000010, 0.000000, 0.000..."
10,17,9,"[0.000000, 0.000000, 0.000010, 0.000000, 0.000..."
15,2,16,"[0.000000, 0.000000, 0.952800, 0.000000, 0.000..."
...,...,...,...
8192,8,26,"[0.000000, 0.000000, 0.030810, 0.000000, 0.000..."
8205,13,9,"[0.108200, 0.008050, 0.018000, 0.000150, 0.000..."
8209,2,27,"[0.000000, 0.000000, 0.963890, 0.000030, 0.000..."
8210,16,8,"[0.000000, 0.000000, 0.434300, 0.000000, 0.000..."


In [22]:
## add columns for predicted probability and predicted probability of actual class
predicted_class_probability=[]
actual_class_probability=[]
for index, row in incorrect_df.iterrows():
  p=row['y_predictions']
  a=row['y_actual']
  pred_proba=row['y_rounded_proba'][p]
  act_proba=row['y_rounded_proba'][a]
  predicted_class_probability.append(pred_proba)
  actual_class_probability.append(act_proba)

In [23]:
incorrect_df['predicted_class_probability']=predicted_class_probability
incorrect_df['actual_class_probability']=actual_class_probability
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability
4,9,14,"[0.010330, 0.000220, 0.002390, 0.050820, 0.000...",0.326090,0.039580
6,0,13,"[0.590350, 0.000450, 0.083620, 0.000120, 0.000...",0.590350,0.050670
8,13,12,"[0.011090, 0.000720, 0.000010, 0.000000, 0.000...",0.514530,0.221130
10,17,9,"[0.000000, 0.000000, 0.000010, 0.000000, 0.000...",0.872480,0.125490
15,2,16,"[0.000000, 0.000000, 0.952800, 0.000000, 0.000...",0.952800,0.038260
...,...,...,...,...,...
8192,8,26,"[0.000000, 0.000000, 0.030810, 0.000000, 0.000...",0.469780,0.191810
8205,13,9,"[0.108200, 0.008050, 0.018000, 0.000150, 0.000...",0.528300,0.041020
8209,2,27,"[0.000000, 0.000000, 0.963890, 0.000030, 0.000...",0.963890,0.004290
8210,16,8,"[0.000000, 0.000000, 0.434300, 0.000000, 0.000...",0.473390,0.001480


In [26]:
#find differences in predicted vs actual probability
incorrect_df['difference']=(incorrect_df['predicted_class_probability'].astype(float))-(incorrect_df['actual_class_probability'].astype(float))
incorrect_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
4,9,14,"[0.010330, 0.000220, 0.002390, 0.050820, 0.000...",0.326090,0.039580,0.28651
6,0,13,"[0.590350, 0.000450, 0.083620, 0.000120, 0.000...",0.590350,0.050670,0.53968
8,13,12,"[0.011090, 0.000720, 0.000010, 0.000000, 0.000...",0.514530,0.221130,0.29340
10,17,9,"[0.000000, 0.000000, 0.000010, 0.000000, 0.000...",0.872480,0.125490,0.74699
15,2,16,"[0.000000, 0.000000, 0.952800, 0.000000, 0.000...",0.952800,0.038260,0.91454
...,...,...,...,...,...,...
8192,8,26,"[0.000000, 0.000000, 0.030810, 0.000000, 0.000...",0.469780,0.191810,0.27797
8205,13,9,"[0.108200, 0.008050, 0.018000, 0.000150, 0.000...",0.528300,0.041020,0.48728
8209,2,27,"[0.000000, 0.000000, 0.963890, 0.000030, 0.000...",0.963890,0.004290,0.95960
8210,16,8,"[0.000000, 0.000000, 0.434300, 0.000000, 0.000...",0.473390,0.001480,0.47191


In [27]:
# ## get counts for each misclassified class
# missed_counts=incorrect_df['y_actual'].value_counts()
# missed_counts

In [28]:
table13=incorrect_df.loc[incorrect_df['y_actual']==13]

In [29]:
table13

,y_predictions,y_actual,y_rounded_proba,predicted_class_probability,actual_class_probability,difference
6,0,13,"[0.590350, 0.000450, 0.083620, 0.000120, 0.000...",0.590350,0.050670,0.53968
28,16,13,"[0.001250, 0.000440, 0.157340, 0.000000, 0.000...",0.330400,0.118560,0.21184
34,9,13,"[0.000060, 0.000000, 0.003550, 0.004970, 0.000...",0.490820,0.390810,0.10001
56,17,13,"[0.011910, 0.000020, 0.011500, 0.004450, 0.000...",0.541930,0.245560,0.29637
182,12,13,"[0.000030, 0.000000, 0.000000, 0.000000, 0.000...",0.894730,0.000870,0.89386
...,...,...,...,...,...,...
7994,12,13,"[0.055390, 0.000100, 0.013310, 0.000000, 0.000...",0.793580,0.079570,0.71401
8014,12,13,"[0.049550, 0.000140, 0.006840, 0.000000, 0.000...",0.797750,0.135910,0.66184
8079,24,13,"[0.000000, 0.000050, 0.000040, 0.000000, 0.000...",0.547280,0.452450,0.09483
8088,9,13,"[0.000000, 0.000000, 0.000000, 0.000000, 0.000...",0.983310,0.015860,0.96745


In [30]:
## find average difference
avg=(table13['difference'].sum())/len(table13['difference'])
avg

0.5207695161290322

Create Weights for underepresented classes

In [6]:
#Manually adjusting weights for only the poorly classified classes
adjusted_class_weights={
    0: 1.,
    1: 1.,
    2: 1.,
    3: 1.,
    4: 1.,
    5: 1.,
    6: 1.,
    7: 1.,
    8: 1.,
    9: 1.,
    10: 1.,
    11: 1.,
    12: 1.,
    13: 1.52,
    14: 1.,
    15: 1.,
    16: 1.,
    17: 1.,
    18: 1.,
    19: 1.,
    20: 1.,
    21: 1.,
    22: 1.,
    23: 1.,
    24: 1.,
    25: 1.,
    26: 1.,
    27: 1.}

Add weights to training

In [6]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model4 = keras.models.Sequential()
model4.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model4.add(MaxPooling2D((2, 2),padding='same'))
model4.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.28))
model4.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.35))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.40))
model4.add(Flatten())
model4.add(Dense(400, activation='relu'))               
model4.add(Dense(28, activation='softmax'))

In [7]:

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model4.fit(X_train, y_train, batch_size=90, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
368/368 [==============================] - 45s 122ms/step - loss: 0.4332 - accuracy: 0.8567 - val_loss: 0.4984 - val_accuracy: 0.8425
Epoch 2/30
368/368 [==============================] - 43s 118ms/step - loss: 0.4341 - accuracy: 0.8571 - val_loss: 0.5248 - val_accuracy: 0.8443
Epoch 3/30
368/368 [==============================] - 43s 118ms/step - loss: 0.4152 - accuracy: 0.8635 - val_loss: 0.5183 - val_accuracy: 0.8425
Epoch 4/30
368/368 [==============================] - 43s 118ms/step - loss: 0.4155 - accuracy: 0.8627 - val_loss: 0.5375 - val_accuracy: 0.8386
Epoch 5/30
368/368 [==============================] - 43s 118ms/step - loss: 0.4209 - accuracy: 0.8622 - val_loss: 0.5562 - val_accuracy: 0.8322
Epoch 6/30
368/368 [==============================] - 43s 118ms/step - loss: 0.4125 - accuracy: 0.8652 - val_loss: 0.5450 - val_accuracy: 0.8351
Epoch 7/30
368/368 [==============================] - 43s 118ms/step - loss: 0.3976 - accuracy: 0.8676 - val_loss: 0.5345 - val_ac

In [ ]:
## weighted all 1's                            Accuracy: 79   Val Accuracy: 79  after 29 epochs
## No weights                                  Accuracy: 79   Val Accuracy: 78  after 37 epochs
## No weights batch size 80                    Accuracy: 85   Val Accuracy: 81  after 30 epochs
## No weights batch size 90                    Accuracy: 88   Val Accuracy: 83.9  after 46 epochs?
## NW, BS 90, last conv2d 256                  Accuracy: 84   Val Accuracy: 83  after 26 epochs
## NW, bs 90, last conv2d 256, dense 400       Accuracy: 88   Val Accuracy  85.29 after 53 epochs

In [10]:
model4.save('drive/MyDrive/CNN_88Acc_84.7Val')

INFO:tensorflow:Assets written to: drive/MyDrive/CNN_88Acc_84.7Val/assets
